In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import json

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_GET_KEY")
client = OpenAI(api_key=api_key)

In [ ]:
# tools = [{
#     "type":"function",
#     "function":{
#         "name":"locate_country_and_winery_based_on_wine",
#         "description":"Retrieve a country and a winery for a given name of wine",
#         "parameters":{
#             "type":"object",
#             "properties":{
#                 "wine_info":{
#                     "type":"string",
#                     "description":"Name of a wine e.g. lodali moscato d'asti"
#                 }
#             },
#             "required":[
#                 "winery_info"
#             ],
#             "additionalProperties":False
#         },
#         "strict":True
#     }
# }]

In [ ]:
# completion = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         {"role":"user", "content":"Where was the wine 'de bortoli deen vat 5' produced?"}
#     ],
#     tools=tools
# )

In [ ]:
# completion_1 = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         {"role":"user", "content":"Where is the Eiffel Tower located?"}
#     ],
#     tools=tools
# )

In [ ]:
# content_tool = completion.choices[0].message.tool_calls
# # content_other = completion_1.choices[0].message.content
# print(content_tool)
# # print(content_other)

[ChatCompletionMessageToolCall(id='call_tt3uqMjXMrAgQbKGr9hlHKyk', function=Function(arguments='{"winery_info":"de bortoli deen vat 5"}', name='locate_country_and_winery_based_on_wine'), type='function')]


In [ ]:
# tool_call = completion.choices[0].message.tool_calls[0]
# args = json.loads(tool_call.function.arguments)
# result = locate_country_and_winery_based_on_wine(args["wine_info"])

In [36]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [37]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"user", "content":"What's the weather like in Paris today?"}
    ],
    tools=tools
)

In [43]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [59]:
messages = [{"role":"user", "content":"What's the weather like in Paris today?"}]

In [62]:
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)
result = get_weather(args["latitude"], args["longitude"])

In [63]:
messages.append(completion.choices[0].message)
messages.append({
    "role":"tool",
    "tool_call_id":tool_call.id,
    "content":str(result)
})

In [51]:
messages.append({
    "role":"user",
    "content":"What is noble rot?"
})

In [64]:
completion_2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools
)

In [65]:
completion_2.choices[0].message.content

'The current temperature in Paris today is 9.4°C.'

In [54]:
completion_2.choices[0].message.content

'Noble rot, scientifically known as Botrytis cinerea, is a type of fungal infection that affects grapes. It is particularly significant in the production of certain sweet wines. The fungus causes the grapes to dehydrate, concentrating the sugars and flavors, which can lead to the creation of rich, sweet wines with complex flavors.\n\nHere are some key points about noble rot:\n\n1. **Weather Conditions**: Noble rot occurs under specific climatic conditions, usually when there is a period of humid weather followed by dry conditions. This allows the Botrytis fungus to develop and then dry out the grapes.\n\n2. **Wine Production**: Wines made from grapes affected by noble rot are typically sweet and can have flavors reminiscent of honey, apricots, and other ripe fruits. Famous examples of these wines include Sauternes from France and Tokaji from Hungary.\n\n3. **Selective Harvesting**: Winemakers often selectively harvest the affected grapes to ensure that only those with the desirable cha